### Указания к работе

- Использовать MRJob
- полезно почитать: https://coderlessons.com/tutorials/bolshie-dannye-i-analitika/izuchit-kartu-umenshit/mapreduce-kratkoe-rukovodstvo 
- Подсказки к 4 и 5 тут - https://www.slideshare.net/Technopark/lecture-05-48215737

In [ ]:
%pip install mrjob
!mkdir -p temp

### Упражнение 1

#### Задача

Дан файл с числами (несколько строк, на которых через пробел расположены числа). Посчитать сумму всех чисел в файле (построчно через MRJob).

Пример:

```
1 2 3
4 5
6
```

Вывод:

```
21
```


#### Решение

In [49]:
%%writefile temp/l4_t1.py
from mrjob.job import MRJob
from mrjob.protocol import TextValueProtocol


class MisterJob(MRJob):
    OUTPUT_PROTOCOL = TextValueProtocol

    def mapper(self, _, line):
        yield from [(None, float(num)) for num in line.split()]

    def reducer(self, _, values):
        yield "sum", str(sum(values))


if __name__ == '__main__':
    MisterJob.run()


Overwriting temp/l4_t1.py


In [50]:
%%writefile temp/l4_t1.txt
9.9 8 7 6
5 4 3
2 1
0

Writing temp/l4_t1.txt


In [51]:
!python temp/l4_t1.py -q temp/l4_t1.txt

45.9


### Упражнение 2

#### Задача 

Дан файл с числами (несколько строк, на которых через пробел расположены числа). Посчитать сумму чисел в файле в каждой строке, а еще и их среднее (построчно через MRJob).

Пример:


```
1 2 3
4 5
6
```

Вывод:

```
6, 2
9, 4.5
6, 6
```

#### Решение

In [32]:
%%writefile temp/l4_t2.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import TextValueProtocol

line_num = 0


def format_float(f: float) -> str:
    return "%d" % f if f.is_integer() else str(f)


class MisterJob(MRJob):
    OUTPUT_PROTOCOL = TextValueProtocol

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper_extract_numbers,
                reducer=self.reducer_calc_stats
            ),
            MRStep(
                reducer=self.reducer_sort_stats
            )
        ]

    def mapper_extract_numbers(self, _, line):
        global line_num
        line_num += 1
        yield from [(line_num, float(i)) for i in line.split()]

    def reducer_calc_stats(self, key, values):
        values = list(values)
        s, l = sum(values), len(values)
        avg = s / l
        yield None, (key, f"{s}, {format_float(avg)}")

    def reducer_sort_stats(self, _, values):
        yield from sorted(values, key=lambda v: v[0])


if __name__ == '__main__':
    MisterJob.run()

Overwriting temp/l4_t2.py


In [52]:
%%writefile temp/l4_t2.txt
9.9 8 7 6
5 4 3
2 1
0

Writing temp/l4_t2.txt


In [53]:
!python temp/l4_t2.py -q temp/l4_t2.txt

30.9, 7.725
12.0, 4
3.0, 1.5
0.0, 0


### Упражнение 3

#### Задача

Дан файл со стоками, в каждой строке слова через пробел. Посчитать самую частую букву.

Пример:

```
aaa bb c
aaa bb
aaa
```

Вывод:

```
9 a
```

#### Решение

In [40]:
%%writefile temp/l4_t3.py
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import TextValueProtocol


class MisterJob(MRJob):
    OUTPUT_PROTOCOL = TextValueProtocol

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper_extract_chars,
                reducer=self.reducer_count_chars
            ),
            MRStep(
                reducer=self.reducer_find_most_common
            )
        ]

    def mapper_extract_chars(self, _, line):
        for word in line.split():
            yield from [(ch, 1) for ch in word]

    def reducer_count_chars(self, key, values):
        yield None, (key, sum(values))

    def reducer_find_most_common(self, _, values):
        ch, count = max(values, key=lambda v: v[1])
        yield None, f"{count} {ch}"


if __name__ == '__main__':
    MisterJob.run()

Overwriting temp/l4_t3.py


In [54]:
%%writefile temp/l4_t3.txt
aaa bb c
aaa bb
aaa

Writing temp/l4_t3.txt


In [55]:
!python temp/l4_t3.py -q temp/l4_t3.txt

9 a


### Упражнение 4.


#### Задача

Есть множество записей. Каждая запись содержит поле F и производное число категорий G={G1, G2,...}
Посчитать общее число уникальных значений поля F для каждой категории.

Пример:

```
Record1: F = 1, G = {a, b}
Record2: F = 2, G = {a, d, e}
Record3: F = 1, G = {b}
Record4: F = 13, G = {a, b}
```

Result:

```
a -> 3 // F =1, F=2, F=3
b -> 2 // F =1, F=3
d -> 1 // F =2
e -> 1 // F =2
```


#### Решение

In [42]:
%%writefile temp/l4_t4.py
import re
from mrjob.job import MRJob
from mrjob.protocol import TextValueProtocol

record_re = re.compile(r'Record\d+: F = (\d+), G = {(.*?)}')
g_re = re.compile(r'(?:, )?([a-z]+)')


class MisterJob(MRJob):
    OUTPUT_PROTOCOL = TextValueProtocol

    def mapper(self, _, line):
        value, categories_str = record_re.findall(line)[0]
        categories = g_re.findall(categories_str)
        for c in categories:
            yield c, int(value)

    def reducer(self, key, values):
        values = list(values)
        fields = ", ".join([f'F={v}' for v in values])
        yield key, f'{key} -> {len(values)} // {fields}'


if __name__ == '__main__':
    MisterJob.run()


Writing temp/l4_t4.py


In [69]:
%%writefile temp/l4_t4.txt
Record1: F = 1, G = {a, b}
Record2: F = 2, G = {a, d, e}
Record3: F = 1, G = {b}
Record4: F = 13, G = {a, b}

Writing temp/l4_t4.txt


In [71]:
!python temp/l4_t4.py -q temp/l4_t4.txt

b -> 3 // F=1, F=1, F=13
d -> 1 // F=2
a -> 3 // F=1, F=2, F=13
e -> 1 // F=2


### Упражнение 5


#### Задача

Есть множество кортежей объектов. Для каждой возможной пары объектов посчитать число картежей, где они встретились вместе. Если число объектов N, то N\*N объектов будет обработано. Если N\*N небольшое, то можно построить матрицу в памяти и реализация довольно простая.


#### Решение

Данное решение предполагает, что объекты представлены в виде строковых имен, а кортеж объектов является последовательностью таких имен, разделенных пробелами.

In [47]:
%%writefile temp/l4_t5.py
from enum import Enum
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import TextValueProtocol


class EventTypes(str, Enum):
    NEW_NAME = 'NEW_NAME'
    NEW_PAIR = 'NEW_PAIR'


class MisterJob(MRJob):
    OUTPUT_PROTOCOL = TextValueProtocol

    def steps(self):
        return [
            MRStep(
                mapper=self.mapper_emit_objects_and_pairs,
                reducer=self.reducer_merge_all_events,
            ),
            MRStep(
                reducer=self.reducer_handle_all_events,
            )
        ]

    def mapper_emit_objects_and_pairs(self, _, line):
        names = line.split()
        for i, n1 in enumerate(names):
            yield EventTypes.NEW_NAME, n1
            for j, n2 in enumerate(names):
                if i > j:
                    yield EventTypes.NEW_PAIR, (n1, n2)

    def reducer_merge_all_events(self, event_type, values):
        yield None, (event_type, list(values))

    def reducer_handle_all_events(self, _, events):
        events = list(events)

        names = next((sorted(set(v))
                      for t, v in events if t == EventTypes.NEW_NAME), [])
        pairs = next((v
                      for t, v in events if t == EventTypes.NEW_PAIR), [])

        matrix = {n: {n: 0 for n in names} for n in names}
        for n1, n2 in pairs:
            matrix[n1][n2] += 1
            if n1 != n2:
                matrix[n2][n1] += 1

        header = "||%s|" % "|".join(names)

        divider = "|%s|" % "|".join([":-:"] * (len(names) + 1))

        matrix_rows = [
            "|%s|%s|" % (name, "|".join(map(str, row.values())))
            for name, row in matrix.items()
        ]

        result = "\n".join([
            header,
            divider,
            *matrix_rows
        ])

        yield None, result


if __name__ == '__main__':
    MisterJob.run()


Overwriting temp/l4_t5.py


In [63]:
%%writefile temp/l4_t5.txt
grass cow grass grass
grass grass
flower grass grass
cow flower cow
grass grass grass

Writing temp/l4_t5.txt


In [67]:
!python temp/l4_t5.py -q temp/l4_t5.txt > ./temp/l4_t5.md
from IPython.display import display, Markdown
display(Markdown("./temp/l4_t5.md"))

||cow|flower|grass|
|:-:|:-:|:-:|:-:|
|cow|1|2|3|
|flower|2|0|2|
|grass|3|2|8|
